# 13.2 最小分散モデル
(Harris & Wolpert, 1998)
minimum-variance model

田中先生の<https://motorcontrol.jp/mc13/MC2019_2_OptimalControlStochastic.pdf>に記載のコードを参考にして作成した。

## 2次計画法
https://www.math.ucla.edu/~wotaoyin/math273a/slides/Lec3_gradient_descent_273a_2015_f.pdf
https://en.wikipedia.org/wiki/Quadratic_programming#Equality_constraints
https://en.wikipedia.org/wiki/Conjugate_gradient_method

In [2]:
using LinearAlgebra

In [3]:
t1 = 224/1000 # time const of eye dynamics
t2 = 13/1000  # another time const of eye dynamics
tm = 10/1000
dt = 1/1000   # simulation time step
tf = 50/1000  # movement duration
tp = 20/1000  # post-movement duration
K = tf/dt
L = tp/dt
x0 = [0; 0; 0]  # initial state
xf = [10; 0; 0] # final state
Ac = [0 1 0; 
    0 0 1; 
    -1/(t1*t2*tm) -1/(t1*t2)-1/(t1*tm)-1/(t2*tm) -1/t1-1/t2-1/tm];
Bc = [0; 0; 1];
A = expm(Ac*dt);
B = inv(Ac)*(eye(3)-expm(Ac*dt))*Bc;
Q = zeros(K+L);

LoadError: UndefVarError: expm not defined

In [ ]:
# calculation of Q
for ell=0:K+L-1
    if ell<K
        for k=K:K+L-1
        tmpQ = A^(k-ell-1)*B*B'*A'^(k-ell-1);
        Q(ell+1, ell+1) = Q(ell+1, ell+1) + tmpQ(1,1);
        end
    else
        for k=ell+1:K+L-1
        tmpQ = A^(k-ell-1)*B*B'*A'^(k-ell-1);
        Q(ell+1, ell+1) = Q(ell, ell) + tmpQ(1,1);
        end
    end
end

In [ ]:
# calculation of C
C = [];
for p=1:L+1
    Ctmp = [];
    for q=1:K+L
        if K-1-(q-1)+(p-1)>0
            Ctmp = [Ctmp A^(K-1-(q-1)+(p-1))*B];
        elseif K-1-(q-1)+(p-1)==0
            Ctmp = [Ctmp B];
        else
            Ctmp = [Ctmp zeros(3,1)];
        end
    end
    C = [C; Ctmp];
end

x = quadprog(H,f,A,b,Aeq,beq) は追加の制約 Aeq*x = beq に従って上記の問題を解きます。Aeq は double 型の行列で、beq は double 型のベクトルです。不等式が存在しない場合は A = [] および b = [] と設定してください。

In [ ]:
# calculation of d
d = [];
for ell=0:L
    d = [d; xf-A^(K+ell)*x0];
end

# solution by quadratic programming
u = quadprog(Q, [], [], [], C, d);

# forward solution
x = zeros(3, K+L);
x(:,1) = x0;
for k=1:K+L-1
    x(:,k+1) = A*x(:,k) + B*u(k);
end

In [ ]:
figure(1);
subplot(211); hold on; plot(x(1,:), 'k', 'linewidth', 2);
set(gca, 'plotboxaspectratio', [1.5 1 1]);
xlabel('time (ms)'); ylabel('eye position (deg)');
subplot(212); hold on; plot(x(2,:), 'k', 'linewidth', 2)
set(gca, 'plotboxaspectratio', [1.5 1 1]);
xlabel('time (ms)'); ylabel('eye velocity (deg/s)');